# SISTEMA RECOMENDADOR

## IMPORTAMOS LAS LIBRERÍAS

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import random

## CARGAMOS LOS DATASETS

df_peliculas: Contiene información sobre películas, incluyendo su ID (movieId), título (title) y géneros (genres).

df_ratings: Contiene las valoraciones de los usuarios, con el ID del usuario (userId), el ID de la película (movieId), la valoración (rating) y una marca de tiempo (timestamp).

In [34]:
df_ratings = pd.read_csv("ratings_f.csv")
df_movies = pd.read_csv("movies.csv")

## Filtrado de Valoraciones Altas

Esta línea de código se utiliza para filtrar las valoraciones del conjunto de datos df_ratings. Selecciona solo aquellas valoraciones que son mayores de 3.5, asumiendo que estas representan películas que a los usuarios les gustaron significativamente. La idea detrás de esto es considerar solo las valoraciones positivas para la generación de reglas de asociación, lo que ayuda a enfocarse en patrones de películas que los usuarios prefirieron.

In [35]:
# Filtrar las valoraciones altas
ratings_altos = df_ratings[df_ratings['rating'] > 3.5]

## Creación de una Matriz Usuario-Película

Inicialización del DataFrame: Se crea un DataFrame llamado usuario_pelicula_matrix utilizando pandas. Este DataFrame tiene una estructura específica:

- Filas: Representan a los usuarios únicos del conjunto de datos df_ratings.
- Columnas: Representan las películas únicas.
- Valores Iniciales: Todos los valores se inicializan en 0.

La idea detrás de esta matriz es crear una representación donde cada celda indica si un usuario (fila) ha calificado positivamente una película (columna).

In [36]:
# Crear un DataFrame donde cada fila es un usuario y cada columna una película
# Inicializar con ceros
usuario_pelicula_matrix = pd.DataFrame(0, index=pd.unique(df_ratings['userId']), columns=pd.unique(df_ratings['movieId']))

Rellenado de la Matriz: Este bucle recorre el DataFrame ratings_altos (que contiene las valoraciones altas) fila por fila.

- Por cada fila (que representa una valoración alta de un usuario para una película específica), se actualiza la matriz usuario_pelicula_matrix.
- Se coloca un 1 en la celda correspondiente a la combinación usuario-película, indicando que este usuario ha calificado positivamente esta película.

In [37]:
# Rellenar la matriz con 1 donde el usuario ha calificado la película positivamente
for index, row in ratings_altos.iterrows():
    usuario_pelicula_matrix.at[row['userId'], row['movieId']] = 1

# Visualizar los primeros datos de la matriz
usuario_pelicula_matrix.head()

169     2471    48516   2571    109487  112552  112556  356     2394    \
1       0       0       1       0       0       0       0       0       0   
2       0       0       0       0       1       1       1       0       0   
3       0       0       0       0       0       0       0       1       1   
4       0       0       0       1       0       0       0       0       0   
5       0       0       0       0       0       1       1       0       0   

   2431    ...  93861   104504  7195    132012  141538  6462    120843  \
1       0  ...       0       0       0       0       0       0       0   
2       0  ...       0       0       0       0       0       0       0   
3       1  ...       0       0       0       0       0       0       0   
4       0  ...       0       0       0       0       0       0       0   
5       0  ...       0       0       0       0       0       0       0   

   121119  8240    97132   
1       0       0       0  
2       0       0       0  
3       0       0       0  
4       0       0       0  
5       0       0       0  

[5 rows x 17292 columns]

## Aplicación del Algoritmo Apriori

Apriori: La función apriori de la biblioteca mlxtend se utiliza para encontrar itemsets frecuentes en el DataFrame usuario_pelicula_matrix.
- usuario_pelicula_matrix: El DataFrame que has preparado, donde cada fila representa un usuario y cada columna representa una película, con 1s y 0s indicando si un usuario ha calificado positivamente una película.
- min_support=0.05: Este parámetro define el soporte mínimo para considerar un itemset como frecuente. En este caso, un itemset (un conjunto de películas) debe aparecer en al menos el 5% de las transacciones (usuarios) para ser considerado frecuente.
- use_colnames=True: Esta opción hace que la función utilice los nombres de las columnas (en este caso, los ID de las películas) en lugar de los índices de columna para identificar los itemsets.

El objetivo de aplicar Apriori es identificar conjuntos de películas que son frecuentemente calificados de manera positiva juntos por diferentes usuarios. Estos itemsets frecuentes son la base para generar reglas de asociación.

In [38]:
# Aplicar el algoritmo Apriori
# min_support es el soporte mínimo para considerar un itemset como frecuente
itemsets_frecuentes = apriori(usuario_pelicula_matrix, min_support=0.05, use_colnames=True, max_len = 2)

# Visualizar los itemsets frecuentes
peliculas_frecuentes = itemsets_frecuentes.sort_values("support", ascending = False).head(10)
peliculas_frecuentes["titulos"] = df_movies[df_movies['movieId'].isin(peliculas_frecuentes.index)]['title'].tolist()

In [39]:
peliculas_frecuentes

support itemsets                             titulos
50  0.267655    (318)                    Toy Story (1995)
12  0.241427    (593)                      Jumanji (1995)
4   0.238925    (296)            Waiting to Exhale (1995)
2   0.231325    (356)                    GoldenEye (1995)
52  0.209082    (260)  Dracula: Dead and Loving It (1995)
1   0.196756   (2571)          Usual Suspects, The (1995)
10  0.196015    (527)               Guardian Angel (1994)
51  0.176830     (50)             Mighty Aphrodite (1995)
67  0.171733    (110)                     Lamerica (1994)
53  0.167377    (858)                     Two Bits (1995)

## Generación de Reglas de Asociación

Generación de Reglas de Asociación: Utilizando la función association_rules de mlxtend, se generan reglas de asociación a partir de los itemsets frecuentes. La métrica utilizada aquí es la "confianza", y el min_threshold de 0.1 indica que solo se considerarán las reglas que tienen al menos un 10% de confianza. La confianza es una medida de la probabilidad de que, cuando se vean ciertas películas (antecedente de la regla), también se vean otras películas (consecuente de la regla).

In [40]:
# Generar reglas de asociación
reglas = association_rules(itemsets_frecuentes, metric="confidence", min_threshold=0.1)

# Visualizar las primeras reglas
reglas.head()

antecedents consequents  antecedent support  consequent support   support  \
0      (2571)       (356)            0.196756            0.231325  0.088323   
1       (356)      (2571)            0.231325            0.196756  0.088323   
2      (2571)        (47)            0.196756            0.141335  0.067933   
3        (47)      (2571)            0.141335            0.196756  0.067933   
4       (296)      (2571)            0.238925            0.196756  0.097312   

   confidence      lift  leverage  conviction  
0    0.448893  1.940527  0.042808    1.394783  
1    0.381811  1.940527  0.042808    1.299350  
2    0.345266  2.442899  0.040125    1.311472  
3    0.480656  2.442899  0.040125    1.546650  
4    0.407292  2.070036  0.050302    1.355211

## Función: recomendar_peliculas

Esta función genera recomendaciones de películas basadas en un conjunto de películas que un usuario ha calificado positivamente.

Parámetros
- peliculas_usuario: Lista de IDs de películas que el usuario ha calificado positivamente.
- reglas: DataFrame de reglas de asociación generadas por el algoritmo Apriori.
- df_peliculas: DataFrame que contiene información sobre las películas, incluyendo sus IDs y títulos.
- numero_recomendaciones (opcional): Número de recomendaciones que se desean generar. Por defecto, es 5.
Proceso
- Traducción de IDs a Títulos: La función comienza traduciendo los IDs de las películas que el usuario ha visto (pasados en peliculas_usuario) a sus respectivos títulos utilizando df_peliculas.

Generación de Recomendaciones Basadas en Reglas:

- Para cada película que el usuario ha visto, la función busca en el conjunto de reglas para encontrar aquellas donde la película está en el antecedente.
- Se extraen los consecuentes de estas reglas y se agregan a un conjunto de recomendaciones. Estos consecuentes son películas que, según las reglas de asociación, tienen una alta probabilidad de ser del interés del usuario dado que ha visto la película del antecedente.
- Filtrado de Películas Ya Vistas: Se eliminan del conjunto de recomendaciones las películas que el usuario ya ha visto, para asegurar que las recomendaciones sean nuevas para el usuario.

- Recomendaciones Aleatorias en Caso de No Encontrar Ninguna: Si no se encuentran recomendaciones a través de las reglas de asociación, la función selecciona un número de películas al azar (exceptuando las que el usuario ya ha visto) para asegurar que siempre se proporcionen recomendaciones.

- Traducción de Recomendaciones a Títulos: Los IDs de las películas recomendadas se traducen a títulos.

- Creación y Retorno del Mensaje de Salida: La función devuelve un mensaje personalizado que lista los títulos de las películas que el usuario ha visto y las recomendaciones generadas.

Esta función es ideal para ser utilizada en un sistema de recomendación interactivo donde los usuarios pueden recibir sugerencias de películas basadas en sus gustos anteriores. Proporciona una mezcla de recomendaciones basadas en datos y recomendaciones aleatorias para garantizar la diversidad y descubrimiento de nuevas películas.



In [41]:
def recomendar_peliculas(peliculas_usuario, reglas, df_peliculas, numero_recomendaciones=5):
    """
    Función para recomendar películas basadas en un conjunto de películas que un usuario ha calificado positivamente.
    
    :param peliculas_usuario: Lista de IDs de películas que el usuario ha calificado positivamente.
    :param reglas: DataFrame de reglas de asociación generadas por Apriori.
    :param df_peliculas: DataFrame con información de las películas.
    :param numero_recomendaciones: Número de recomendaciones a devolver.
    :return: Mensaje con películas vistas y recomendaciones.
    """
    # Traducir ID de películas vistas a títulos
    titulos_vistos = df_peliculas[df_peliculas['movieId'].isin(peliculas_usuario)]['title'].tolist()

    recomendaciones = set()
    for pelicula in peliculas_usuario:
        # Filtrar reglas que tienen la película en el antecedente
        reglas_filtradas = reglas[reglas['antecedents'].apply(lambda x: pelicula in x)]

        # Añadir todas las recomendaciones (consecuentes)
        for index, row in reglas_filtradas.iterrows():
            recomendaciones.update(row['consequents'])

    # Eliminar las películas que el usuario ya ha visto
    recomendaciones.difference_update(peliculas_usuario)

    # Si no hay recomendaciones basadas en reglas, elegir al azar
    if not recomendaciones:
        todas_las_peliculas = set(df_peliculas['movieId']) - set(peliculas_usuario)
        recomendaciones = set(random.sample(todas_las_peliculas, numero_recomendaciones))

    # Traducir ID de películas recomendadas a títulos
    recomendaciones_titulos = df_peliculas[df_peliculas['movieId'].isin(recomendaciones)]['title'].tolist()

    # Crear mensaje de salida
    mensaje = f"Dado que has visto las siguientes películas: {', '.join(titulos_vistos)}, te recomendamos: {', '.join(recomendaciones_titulos[:numero_recomendaciones])}"

    return mensaje


## Ejemplo de Uso de la Función recomendar_peliculas

In [46]:
# Ejemplo de lista de películas calificadas positivamente por un usuario (IDs de películas)
peliculas_usuario = [12, 234] # Asumiendo que estos son IDs de películas

# Llamar a la función de recomendación
mensaje_recomendacion = recomendar_peliculas(peliculas_usuario, reglas, df_movies, 5)

# Imprimir el mensaje de recomendación
print(mensaje_recomendacion)

Dado que has visto las siguientes películas: Dracula: Dead and Loving It (1995), Exit to Eden (1994), te recomendamos: Loaded Weapon 1 (National Lampoon's Loaded Weapon 1) (1993), Shoot to Kill (1988), Black Sunday (1977), Torment (Hets) (1944), Crumbs (2015)
